# Business Undertsanding 

A new business owner wants to open a restuarant in California. He has chosen three cities disctricts to explore.The business owners wants to see the surrounding venues for those cities and what type of restuarants are popular. We will examine the restuarant types and the likes for each type of restuarant. We will see which area what what type of resuarant and the likes for it. 


# Data

Using Foursquare and the raw data scraped from each url. We are only focusing on:

- Name
- Category 
- Latitude
- Longitude
- Id/City

Using another API we will get the __likes__ data and applying that to the machine learning models. 

Comnbined together we will have an overview of all the information from each city and see the type of restuarant has the most likes. 


# Methodology 


The owner wants to predict the 'likes' of a certain type of restuarant. The more likes it can predict the higher chance for success. 

__Libraries needed:__
- Sci-kit learn

__Machine learning models__
- Logistic regression: great for predicting classification. In this case __yes__ like or __no__ like.

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np 
import json
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 
import matplotlib.pyplot as plt
import pylab as pl
import itertools
import warnings
warnings.filterwarnings('ignore')

from urllib.request import urlopen
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim 
from sklearn import linear_model
from sklearn.metrics import jaccard_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, r2_score

print('All libraries have been imported')

All libraries have been imported


In [2]:
address1 = 'Oakland, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location1 = geolocator.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address1, latitude1, longitude1))

address2 = 'Emeryville, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address2, latitude2, longitude2))

address3 = 'San Diego, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location3 = geolocator.geocode(address3)
latitude3 = location3.latitude
longitude3 = location3.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address3, latitude3, longitude3))

The geograpical coordinate of Oakland, California are 37.8044557, -122.2713563.
The geograpical coordinate of Emeryville, California are 37.8314089, -122.2865266.
The geograpical coordinate of San Diego, California are 32.7174202, -117.1627728.


In [3]:
CLIENT_ID = 'FOOE0M5S5QMCVKKXYQZTD2Z55JHNOE0K0Y3SFUXUYTJTLV4Q' # your Foursquare ID
CLIENT_SECRET = '3LUKVDWWTVWBNUEQ5PVDPD5SQB1CBILNUE1HP3WVM44C013M' # your Foursquare Secret
VERSION = '20210804' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URLs
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude1, 
    longitude1, 
    radius, 
    LIMIT)


# create URLs
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude2, 
    longitude2, 
    radius, 
    LIMIT)


# create URLs
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude3, 
    longitude3, 
    radius, 
    LIMIT)

print(url1, url2, url3)

Your credentails:
CLIENT_ID: FOOE0M5S5QMCVKKXYQZTD2Z55JHNOE0K0Y3SFUXUYTJTLV4Q
CLIENT_SECRET:3LUKVDWWTVWBNUEQ5PVDPD5SQB1CBILNUE1HP3WVM44C013M
https://api.foursquare.com/v2/venues/explore?&client_id=FOOE0M5S5QMCVKKXYQZTD2Z55JHNOE0K0Y3SFUXUYTJTLV4Q&client_secret=3LUKVDWWTVWBNUEQ5PVDPD5SQB1CBILNUE1HP3WVM44C013M&v=20210804&ll=37.8044557,-122.2713563&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=FOOE0M5S5QMCVKKXYQZTD2Z55JHNOE0K0Y3SFUXUYTJTLV4Q&client_secret=3LUKVDWWTVWBNUEQ5PVDPD5SQB1CBILNUE1HP3WVM44C013M&v=20210804&ll=37.8314089,-122.2865266&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=FOOE0M5S5QMCVKKXYQZTD2Z55JHNOE0K0Y3SFUXUYTJTLV4Q&client_secret=3LUKVDWWTVWBNUEQ5PVDPD5SQB1CBILNUE1HP3WVM44C013M&v=20210804&ll=32.7174202,-117.1627728&radius=1000&limit=100


In [4]:
# scrape the data from the generated URLs

results1 = requests.get(url1).json()
results1

results2 = requests.get(url2).json()
results2

results3 = requests.get(url3).json()
results3

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

# FIRST CITY   

venues1 = results1['response']['groups'][0]['items']
nearby_venues1 = pd.json_normalize(venues1) # flatten JSON

# filter columns
filtered_columns1 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues1 = nearby_venues1.loc[:, filtered_columns1]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]

# SECOND CITY

venues2 = results2['response']['groups'][0]['items']
nearby_venues2 = pd.json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns2 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues2 = nearby_venues2.loc[:, filtered_columns2]

# filter the category for each row
nearby_venues2['venue.categories'] = nearby_venues2.apply(get_category_type, axis=1)

# clean columns
nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]

# THIRD CITY

venues3 = results3['response']['groups'][0]['items']
nearby_venues3 = pd.json_normalize(venues3) # flatten JSON

# filter columns
filtered_columns3 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues3 = nearby_venues3.loc[:, filtered_columns3]

# filter the category for each row
nearby_venues3['venue.categories'] = nearby_venues3.apply(get_category_type, axis=1)

# clean columns
nearby_venues3.columns = [col.split(".")[-1] for col in nearby_venues3.columns]





print('{} venues for Oakland, California were returned by Foursquare.'.format(nearby_venues1.shape[0]))
print()
print('{} venues for Emeryville, California were returned by Foursquare.'.format(nearby_venues2.shape[0]))
print()
print('{} venues for San Diego, California were returned by Foursquare.'.format(nearby_venues3.shape[0]))

100 venues for Oakland, California were returned by Foursquare.

100 venues for Emeryville, California were returned by Foursquare.

100 venues for San Diego, California were returned by Foursquare.


In [5]:
# add locations data to the data sets of each city

nearby_venues1['city'] = 'Oakland'
nearby_venues2['city'] = 'Emeryville'
nearby_venues3['city'] = 'San Diego'

In [6]:
nearby_venues1.categories.value_counts()

Bar                              7
Vietnamese Restaurant            5
Coffee Shop                      5
Café                             5
Chinese Restaurant               5
Sandwich Place                   4
Beer Garden                      3
Mexican Restaurant               3
Clothing Store                   2
Bubble Tea Shop                  2
Caribbean Restaurant             2
Burger Joint                     2
Sports Bar                       2
Wine Bar                         2
Dim Sum Restaurant               2
Vegetarian / Vegan Restaurant    2
Bakery                           2
Ice Cream Shop                   2
Brewery                          2
Japanese Restaurant              2
Music Venue                      2
Brazilian Restaurant             2
Juice Bar                        1
Yoga Studio                      1
Comic Shop                       1
Breakfast Spot                   1
American Restaurant              1
Indian Restaurant                1
Tiki Bar            

** Oakland: 
__Top 3 venues__
- Bar
- Vietnamese restuarant 
- Chinese restuarant 

In [7]:
nearby_venues2.categories.value_counts()

Clothing Store                     7
Coffee Shop                        6
Cosmetics Shop                     5
Furniture / Home Store             4
Pizza Place                        4
Accessories Store                  2
Taco Place                         2
Movie Theater                      2
Pet Store                          2
Bakery                             2
Park                               2
Electronics Store                  2
Mobile Phone Shop                  2
Mattress Store                     2
Candy Store                        2
Kids Store                         2
Diner                              2
Scandinavian Restaurant            2
Discount Store                     2
Sandwich Place                     2
Bar                                2
Burger Joint                       1
Casino                             1
Southern / Soul Food Restaurant    1
Supplement Shop                    1
Big Box Store                      1
Arts & Crafts Store                1
P

**Emeryville: Top 3 ranking

- Clothing
- Coffee
- Cosmetic 

In [8]:
nearby_venues3.categories.value_counts()

Hotel                      12
Italian Restaurant          6
Mexican Restaurant          5
Bar                         5
Coffee Shop                 4
Sushi Restaurant            3
Café                        3
Sandwich Place              3
New American Restaurant     3
Music Venue                 2
American Restaurant         2
Pub                         2
Seafood Restaurant          2
Theater                     2
Gym                         2
Brewery                     2
Hookah Bar                  2
Ice Cream Shop              2
Cosmetics Shop              1
Juice Bar                   1
Beer Store                  1
Donut Shop                  1
Business Service            1
Chocolate Shop              1
Dessert Shop                1
Brazilian Restaurant        1
Indian Restaurant           1
Tattoo Parlor               1
Vietnamese Restaurant       1
Turkish Restaurant          1
Theme Restaurant            1
Lingerie Store              1
Massage Studio              1
Tiki Bar  

**San Diego: Top 3 ranking 

- Hotel
- Italian Restuarant 
- Mexican 

In [9]:
# combine the three cities into one data set

nearby_venues = nearby_venues1.copy()
nearby_venues = nearby_venues.append(nearby_venues2)
nearby_venues = nearby_venues.append(nearby_venues3)

In [10]:
nearby_venues

,name,categories,lat,lng,id,city
0,Oaklandish,Clothing Store,37.805075,-122.270726,4dfb9c2c1f6eeef806ab898c,Oakland
1,Cafe Van Kleef,Bar,37.806660,-122.270273,46884818f964a52056481fe3,Oakland
2,Bar Shiru,Bar,37.806378,-122.270393,5c5b9abdf870fd002c35d291,Oakland
3,Cape & Cowl,Comic Shop,37.806725,-122.272747,56562410498ea43ab630819a,Oakland
4,The Hatch,Bar,37.805006,-122.269584,5482a50d498e9b7b77f386ba,Oakland
5,Woods Bar & Brewery,Brewery,37.806889,-122.270415,5419f32c498e561ee5c2fa38,Oakland
6,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Oakland
7,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Oakland
8,Lucky Duck Bicycle Cafe,Café,37.801684,-122.268656,57bf810c498ee0a34d9f8ca1,Oakland
9,Ume Yoga,Yoga Studio,37.805493,-122.270945,57348ac1498e7aae9899a535,Oakland


In [11]:
nearby_venues.city.value_counts()

Emeryville    100
Oakland       100
San Diego     100
Name: city, dtype: int64

In [12]:
nearby_venues['categories'].unique()

array(['Clothing Store', 'Bar', 'Comic Shop', 'Brewery',
       'Japanese Restaurant', 'Vegetarian / Vegan Restaurant', 'Café',
       'Yoga Studio', 'Music Venue', 'Bagel Shop',
       'Vietnamese Restaurant', 'Tiki Bar', 'Coffee Shop', 'Wine Bar',
       'Mexican Restaurant', 'Seafood Restaurant', 'Dance Studio',
       'Chinese Restaurant', 'Brazilian Restaurant',
       'Kitchen Supply Store', 'Caribbean Restaurant', 'Beer Bar',
       'Nightclub', 'Cocktail Bar', 'Food Court', 'Sushi Restaurant',
       'Sausage Shop', 'Taco Place', 'Ice Cream Shop',
       'Gym / Fitness Center', 'Cupcake Shop', 'Dessert Shop',
       'Tapas Restaurant', 'Bubble Tea Shop', 'Cambodian Restaurant',
       'Burger Joint', 'Bakery', 'Sandwich Place', 'Climbing Gym',
       'Juice Bar', 'American Restaurant', 'Tea Room', 'Lounge',
       'Farmers Market', 'New American Restaurant', 'Gay Bar',
       'Beer Garden', 'Thai Restaurant', 'Arts & Crafts Store',
       'Indian Restaurant', 'Dim Sum Restauran

## Let us focus on restuarants 

In [13]:
# check list and manually remove all non-restaurant data

nearby_venues['categories'].unique()

removal_list = ['Clothing Store','Bar','Brewery', 
                'Comic Shop', 'Yoga Studio','Café', 
                'Coffee Shop', 'Tiki Bar', 'Music Venue', 
                'Wine Bar',  'Cocktail Bar', 'Dance Studio', 
                'Gym / Fitness Center','Beer Bar', 
                'Bubble Tea Shop', 'Nightclub', 'Food Court', 
                'Ice Cream Shop', 'Cupcake Shop', 'Skating Rink', 
                'Dessert Shop', 'Climbing Gym', 'Bakery', 
                'Farmers Market', 'Gay Bar','Beer Garden',
                'Tea Room','Arts & Crafts Store', 'Grocery Store', 
                'Sports Bar', 'Museum', 'Street Food Gathering', 
                'Library', 'Skate Park', 'Movie Theater','Park', 
                'Gym', 'Stadium', 'Furniture / Home Store', 'Discount Store', 
                'Playground', 'Cosmetics Shop', 'Casino', 
                'Pet Store','Electronics Store', 'Snack Place',
                'Salon / Barbershop', 'Shopping Plaza', 'Deli / Bodega', 
                'Candy Store', 'Liquor Store', 'Hotel', 
                'Shoe Store', 'Bookstore', 'Shopping Mall', 
                'Dive Bar', 'Video Game Store', 'Pharmacy', 
                'Accessories Store', 'Lingerie Store', 'Mobile Phone Shop', 
                'Pool Hall', 'Juice Bar', 'Kids Store', 
                'Supplement Shop', 'Big Box Store', 'Mattress Store', 
                'Hardware Store', 'Paper / Office Supplies Store', 'Theater', 
                'Business Service', 'Donut Shop', 'Beer Store', 
                'Lounge', 'Health Food Store', 'Pedestrian Plaza', 
                'Hookah Bar', 'Concert Hall', 'Chocolate Shop', 
                'Hostel', 'Convenience Store', 'Pub', 
                'Plaza', 'Comedy Club', 'Speakeasy', 
                'Tattoo Parlor', 'Massage Studio']

nearby_venues = nearby_venues[~nearby_venues['categories'].isin(removal_list)]

nearby_venues['categories'].unique().tolist()

['Japanese Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Bagel Shop',
 'Vietnamese Restaurant',
 'Mexican Restaurant',
 'Seafood Restaurant',
 'Chinese Restaurant',
 'Brazilian Restaurant',
 'Kitchen Supply Store',
 'Caribbean Restaurant',
 'Sushi Restaurant',
 'Sausage Shop',
 'Taco Place',
 'Tapas Restaurant',
 'Cambodian Restaurant',
 'Burger Joint',
 'Sandwich Place',
 'American Restaurant',
 'New American Restaurant',
 'Thai Restaurant',
 'Indian Restaurant',
 'Dim Sum Restaurant',
 'Dumpling Restaurant',
 'Breakfast Spot',
 'Falafel Restaurant',
 'Diner',
 'Pizza Place',
 'Mediterranean Restaurant',
 'Scandinavian Restaurant',
 'Southern / Soul Food Restaurant',
 'Filipino Restaurant',
 'Asian Restaurant',
 'Food Truck',
 'Wings Joint',
 'Burrito Place',
 'Fast Food Restaurant',
 "Men's Store",
 'Italian Restaurant',
 'Theme Restaurant',
 'Hot Dog Joint',
 'Turkish Restaurant',
 'Gastropub',
 'Empanada Restaurant']

## We will gather data about each type of restuarant based on its number of likes 

In [14]:
# set up to pull the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in list(nearby_venues.id):
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)


nearby_venues['likes'] = like_list
nearby_venues

[154, 77, 22, 72, 199, 63, 50, 367, 39, 115, 100, 51, 6, 176, 182, 43, 40, 171, 46, 24, 39, 91, 52, 33, 237, 76, 50, 261, 25, 13, 15, 36, 38, 5, 45, 97, 42, 66, 226, 52, 9, 111, 247, 191, 329, 127, 132, 36, 24, 13, 31, 7, 55, 61, 92, 16, 18, 22, 63, 17, 4, 65, 5, 1, 17, 1, 0, 3, 31, 156, 76, 3, 79, 33, 171, 26, 131, 41, 35, 15, 94, 142, 105, 24, 18, 34, 35, 297, 31, 21, 31, 222, 129, 97, 185, 104, 323, 264, 480, 132, 30, 103, 546, 31, 19, 8]


,name,categories,lat,lng,id,city,likes
6,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Oakland,154
7,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Oakland,77
11,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Oakland,22
12,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Oakland,72
16,Cosecha,Mexican Restaurant,37.801607,-122.274889,4e179c7752b123a586cef176,Oakland,199
17,The Cook And Her Farmer,Seafood Restaurant,37.801583,-122.274860,5376a879498e8eeb2402cd71,Oakland,63
20,Binh Minh Quan,Vietnamese Restaurant,37.802020,-122.269396,4a75022cf964a52043e01fe3,Oakland,50
23,Shan Dong,Chinese Restaurant,37.800500,-122.270046,4a22d464f964a520927d1fe3,Oakland,367
24,Galeto Brazilian Steakhouse,Brazilian Restaurant,37.803117,-122.275170,535a7936498e249406263166,Oakland,39
26,Umami Mart,Kitchen Supply Store,37.800483,-122.273744,501c7838e4b08947b4df755e,Oakland,115


In [15]:
nearby_venues.count()

name          106
categories    106
lat           106
lng           106
id            106
city          106
likes         106
dtype: int64

In [16]:
raw_dataset = nearby_venues
raw_dataset.head()

,name,categories,lat,lng,id,city,likes
6,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Oakland,154
7,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Oakland,77
11,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Oakland,22
12,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Oakland,72
16,Cosecha,Mexican Restaurant,37.801607,-122.274889,4e179c7752b123a586cef176,Oakland,199


In [17]:
# Find unique restuarants

raw_dataset['categories'].unique().tolist()

['Japanese Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Bagel Shop',
 'Vietnamese Restaurant',
 'Mexican Restaurant',
 'Seafood Restaurant',
 'Chinese Restaurant',
 'Brazilian Restaurant',
 'Kitchen Supply Store',
 'Caribbean Restaurant',
 'Sushi Restaurant',
 'Sausage Shop',
 'Taco Place',
 'Tapas Restaurant',
 'Cambodian Restaurant',
 'Burger Joint',
 'Sandwich Place',
 'American Restaurant',
 'New American Restaurant',
 'Thai Restaurant',
 'Indian Restaurant',
 'Dim Sum Restaurant',
 'Dumpling Restaurant',
 'Breakfast Spot',
 'Falafel Restaurant',
 'Diner',
 'Pizza Place',
 'Mediterranean Restaurant',
 'Scandinavian Restaurant',
 'Southern / Soul Food Restaurant',
 'Filipino Restaurant',
 'Asian Restaurant',
 'Food Truck',
 'Wings Joint',
 'Burrito Place',
 'Fast Food Restaurant',
 "Men's Store",
 'Italian Restaurant',
 'Theme Restaurant',
 'Hot Dog Joint',
 'Turkish Restaurant',
 'Gastropub',
 'Empanada Restaurant']

In [18]:
# we can group some cuisines together to make a better categorical variable

european = ['Mediterranean Restaurant', 'Scandinavian Restaurant', 'Pizza Place',
       'French Restaurant', 'Falafel Restaurant', 'Italian Restaurant',
       'Turkish Restaurant']

latin = ['Mexican Restaurant', 'Taco Place', 'Brazilian Restaurant', 
          'Burrito Place']

asian = ['Japanese Restaurant', 'Vietnamese Restaurant', 'Chinese Restaurant',
         'Hot Dog Joint', 'Hotpot Restaurant', 'Indian Restaurant',
         'Thai Restaurant', 'Dumpling Restaurant', 'Dim Sum Restaurant',
         'Asian Restaurant', 'Filipino Restaurant', 'Sushi Restaurant',
         'Ramen Restaurant']

american = ['Vegetarian / Vegan Restaurant', 'Seafood Restaurant', 'Caribbean Restaurant',
           'Burger Joint', 'American Restaurant', 'New American Restaurant',
            'Southern / Soul Food Restaurant', 'Diner']

casual = ['Bagel Shop', 'Sandwich Place', 'Fried Chicken Joint', 
          'Breakfast Spot', 'Wings Joint', 'Fast Food Restaurant',
          'Theme Restaurant']

def conditions(s):
    if s['categories'] in european:
        return 'european'
    if s['categories'] in latin:
        return 'latin'
    if s['categories'] in asian:
        return 'asian'
    if s['categories'] in american:
        return 'american'
    if s['categories'] in casual:
        return 'casual'

raw_dataset['categories_classified'] = raw_dataset.apply(conditions, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified
6,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Oakland,154,asian
7,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Oakland,77,american
11,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Oakland,22,casual
12,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Oakland,72,asian
16,Cosecha,Mexican Restaurant,37.801607,-122.274889,4e179c7752b123a586cef176,Oakland,199,latin
17,The Cook And Her Farmer,Seafood Restaurant,37.801583,-122.274860,5376a879498e8eeb2402cd71,Oakland,63,american
20,Binh Minh Quan,Vietnamese Restaurant,37.802020,-122.269396,4a75022cf964a52043e01fe3,Oakland,50,asian
23,Shan Dong,Chinese Restaurant,37.800500,-122.270046,4a22d464f964a520927d1fe3,Oakland,367,asian
24,Galeto Brazilian Steakhouse,Brazilian Restaurant,37.803117,-122.275170,535a7936498e249406263166,Oakland,39,latin
26,Umami Mart,Kitchen Supply Store,37.800483,-122.273744,501c7838e4b08947b4df755e,Oakland,115,None


In [19]:
# double check to make sure categories_classified has been created correctly

pd.crosstab(index = raw_dataset["categories_classified"],
            columns="count")

col_0,count
categories_classified,
american,22
asian,26
casual,16
european,17
latin,17


In [20]:
raw_dataset['likes'].mean()

89.5754716981132

In [21]:
# create a function to bin for us

def rankings(df):
    
    if df['likes'] <= 60:
        return 3
    
    elif df['likes'] <= 100:
        return 2
    
    elif df['likes'] > 100:
        return 1

In [22]:
# apply rankings function to dataset

raw_dataset['ranking'] = raw_dataset.apply(rankings, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified,ranking
6,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Oakland,154,asian,1
7,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Oakland,77,american,2
11,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Oakland,22,casual,3
12,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Oakland,72,asian,2
16,Cosecha,Mexican Restaurant,37.801607,-122.274889,4e179c7752b123a586cef176,Oakland,199,latin,1
17,The Cook And Her Farmer,Seafood Restaurant,37.801583,-122.274860,5376a879498e8eeb2402cd71,Oakland,63,american,2
20,Binh Minh Quan,Vietnamese Restaurant,37.802020,-122.269396,4a75022cf964a52043e01fe3,Oakland,50,asian,3
23,Shan Dong,Chinese Restaurant,37.800500,-122.270046,4a22d464f964a520927d1fe3,Oakland,367,asian,1
24,Galeto Brazilian Steakhouse,Brazilian Restaurant,37.803117,-122.275170,535a7936498e249406263166,Oakland,39,latin,3
26,Umami Mart,Kitchen Supply Store,37.800483,-122.273744,501c7838e4b08947b4df755e,Oakland,115,None,1


In [23]:
# create dummies for linear regression modelling

# one hot encoding
reg_dataset = pd.get_dummies(raw_dataset[['categories_classified', 
                                          'city',]], 
                               prefix="", 
                               prefix_sep="")

# add name, ranking, and likes columns back to dataframe
reg_dataset['ranking'] = raw_dataset['ranking']
reg_dataset['likes'] = raw_dataset['likes']
reg_dataset['name'] = raw_dataset['name']

# move name column to the first column
reg_columns = [reg_dataset.columns[-1]] + list(reg_dataset.columns[:-1])
reg_dataset = reg_dataset[reg_columns]


reg_dataset.head()

,name,american,asian,casual,european,latin,Emeryville,Oakland,San Diego,ranking,likes
6,Abura-Ya,0,1,0,0,0,0,1,0,1,154
7,Golden Lotus Vegetarian Restaurant,1,0,0,0,0,0,1,0,2,77
11,Beauty’s Bagel Shop,0,0,1,0,0,0,1,0,3,22
12,Tay Ho Restaurant & Bar,0,1,0,0,0,0,1,0,2,72
16,Cosecha,0,0,0,0,1,0,1,0,1,199


In [24]:
# Multiple Linear Regression

msk = np.random.rand(len(reg_dataset)) < 0.8
train = reg_dataset[msk]
test = reg_dataset[~msk]

regr = linear_model.LinearRegression()
x = np.asanyarray(train[['american', 'asian', 'casual',
                         'european', 'latin', 'Oakland', 
                         'Emeryville', 'San Diego']])

y = np.asanyarray(train[['likes']])
regr.fit (x, y)

# The coefficients

print ('Coefficients: ', regr.coef_)

Coefficients:  [[ 12.25718815 -65.87614598 -66.43461125 -50.65007539  -9.61975749
   17.51578708 -42.98738719  25.47160011]]


In [25]:
# Multiple Linear Regression Prediction Capabilities

y_hat= regr.predict(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Oakland', 
                         'Emeryville', 'San Diego']])

x = np.asanyarray(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Oakland', 
                         'Emeryville', 'San Diego']])

y = np.asanyarray(test[['likes']])
print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

Residual sum of squares: 10227.82
Variance score: 0.12


In [26]:
# Multinomial Ordinal Logistic Regression

x_train = np.asanyarray(train[['american', 'asian', 'casual',
                         'european', 'latin', 'Oakland', 
                         'Emeryville', 'San Diego']])

y_train = np.asanyarray(train['ranking'])

x_test = np.asanyarray(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Oakland', 
                         'Emeryville', 'San Diego']])

y_test = np.asanyarray(test['ranking'])


mul_ordinal = linear_model.LogisticRegression(multi_class='multinomial',
                                              solver='newton-cg',
                                              fit_intercept=True).fit(x_train,
                                                                      y_train)

mul_ordinal

coef = mul_ordinal.coef_[0]
print (coef)

[ 0.07692756 -0.69266886 -0.49172437 -0.09491516 -0.29421969  0.22297706
 -0.56619666  0.34324736]


In [27]:
# Multinomial Ordinal Logistic Regression Prediction Capabilities

yhat = mul_ordinal.predict(x_test)
yhat

yhat_prob = mul_ordinal.predict_proba(x_test)
yhat_prob

jaccard_score(y_test, yhat, average='weighted')

0.5064761904761905

In [28]:
log_loss(y_test, yhat_prob)

0.7908752706986969

In [29]:
# Exploration of Coefficient Magnitudes of Full Dataset

x_all = np.asanyarray(reg_dataset[['american', 'asian', 'casual',
                                   'european', 'latin', 'Oakland', 
                                   'Emeryville', 'San Diego']])

y_all = np.asanyarray(reg_dataset['ranking'])



LR = linear_model.LogisticRegression(multi_class='multinomial',
                                            solver='newton-cg',
                                            fit_intercept=True).fit(x_all,
                                                                    y_all)

LR

coef = LR.coef_[0]
print(coef)

[ 0.22941597 -0.67064718 -0.50554341 -0.03019517 -0.23368954  0.13834373
 -0.57916854  0.44082532]


In [30]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           1       0.30      1.00      0.46         3
           2       0.00      0.00      0.00         3
           3       0.87      0.68      0.76        19

    accuracy                           0.64        25
   macro avg       0.39      0.56      0.41        25
weighted avg       0.69      0.64      0.64        25

